In [1]:
import pandas as pd
from fedas_classifier import FedasClassifier
fc = FedasClassifier()

raw_train = pd.read_csv('data_technical_test/train_technical_test.csv', 
    na_values="",
    dtype={
        "incorrect_fedas_code": object, 
        "correct_fedas_code": object, 
    },
    parse_dates=["avalability_start_date", "avalability_end_date"])

train = raw_train.copy(deep=True).drop(columns=["correct_fedas_code"])
target = raw_train["correct_fedas_code"]

fc.fit(train, target)
fc.predict(train[:20])

Normalizing features...
Normalizing target...
Fitting model...
Done.
Predicting fedas codes...
Normalizing features...


,fedas,confidence
0,378101,0.822763
1,364308,0.940800
2,175890,0.552432
3,224118,1.000000
4,115944,0.133752
5,200124,0.805780
6,175850,1.000000
7,375952,0.245578
8,315933,0.568420
9,141791,0.564114


In [2]:
comparison = pd.concat((fc.predict(train[:20]), target[:20]), axis=1)
comparison['correct'] = comparison['fedas'] == comparison['correct_fedas_code']
comparison

Predicting fedas codes...
Normalizing features...


,fedas,confidence,correct_fedas_code,correct
0,378101,0.822763,378101,True
1,364308,0.940800,364308,True
2,175890,0.552432,175890,True
3,224118,1.000000,224118,True
4,115944,0.133752,115944,True
5,200124,0.805780,200124,True
6,175850,1.000000,175850,True
7,375952,0.245578,375022,False
8,315933,0.568420,315933,True
9,141791,0.564114,137791,False


In [3]:
comparison[comparison['correct'] == False]

,fedas,confidence,correct_fedas_code,correct
7,375952,0.245578,375022,False
9,141791,0.564114,137791,False
10,200127,0.049403,200621,False


In [4]:
test = pd.read_csv('data_technical_test/test_technical_test.csv', 
    na_values="",
    dtype={
        "incorrect_fedas_code": object, 
        "correct_fedas_code": object, 
    },
    parse_dates=["avalability_start_date", "avalability_end_date"])

fc.predict(test)

Predicting fedas codes...
Normalizing features...
